# Camera Trap Processing Thesis

Author: Jameson Andrews\
EDITS:\
09/13/2023, Initial Creation, Jameson

## Load model

In [27]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras import models
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19


# testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
testPicture = load_img('../DeerNight.jpg', target_size=(224,224))

pictureArray = img_to_array(testPicture)
# reshape data for the model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# load the model



model = VGG19()
model.trainable = False
model.summary()



# predict the probability across all output classes
# yhat = model.predict(pictureArray)
# convert the probabilities to class labels
# labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
# label = labels[0][0]
# print the classification
# print(labels)

1/1 [==============================] - 0s 186ms/step
[[('n02395406', 'hog', 0.14550796), ('n02797295', 'barrow', 0.040324006), ('n03393912', 'freight_car', 0.024490995), ('n02437616', 'llama', 0.02321001), ('n02793495', 'barn', 0.021253262)]]
Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                

2023-09-20 21:39:43.649052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


## Try with a different input shape

In [17]:
# testPicture = load_img('../deerdownload.jpg', target_size=(2592, 4608))
testPicture = load_img('../DeerNight.jpg')
pictureArray = img_to_array(testPicture)
# print (pictureArray.shape)
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
pictureArray = preprocess_input(pictureArray)



# load model and specify a new input shape for images
new_input = keras.Input(shape=(2592, 4608, 3))
newModel = VGG19(include_top=False, input_tensor=new_input, pooling="maz")
newModel.trainable = False
newModel.summary()



Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 2592, 4608, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, 2592, 4608, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 2592, 4608, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 1296, 2304, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, 1296, 2304, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 1296, 2304, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 648, 1152, 128)    0     

## Create new Output layer

In [66]:
from keras.layers import Dense, Activation, Flatten

completeModel = models.Sequential()
completeModel.add(model)
# Flatten if using newModel
# completeModel.add(Flatten())
completeModel.add(Dense(100, activation = "relu"))
completeModel.add(Dense(2, activation = "softmax"))
completeModel.summary()
completeModel.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 1000)              143667240 
                                                                 
 dense_35 (Dense)            (None, 100)               100100    
                                                                 
 dense_36 (Dense)            (None, 2)                 202       
                                                                 
Total params: 143,767,542
Trainable params: 100,302
Non-trainable params: 143,667,240
_________________________________________________________________


## Orgainze test and training data

In [100]:
import os
categories = [1] * 756 + [0] * 357
# for root, dirs, files in os.walk("trainingData", topdown=False):
#     print (root, dirs, files)
#     if root == "trainingData/deerTrain":
#         categories.append(1)
#     elif dirs == "trainingData/notDeerTrain":
#         categories.append(0)
# print(categories)
trainImages = keras.utils.image_dataset_from_directory(
    directory='trainingData',
    labels=categories,
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))
testImages = keras.utils.image_dataset_from_directory(
    directory='testData',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(224, 224))

Found 1113 files belonging to 2 classes.
Found 316 files belonging to 2 classes.


## Train Last three layers

In [102]:
completeModel.fit(trainImages, epochs=10, batch_size=100)

Epoch 1/10


35/35 [==============================] - 13s 365ms/step - loss: 0.2202 - accuracy: 0.6792
Epoch 2/10
35/35 [==============================] - 13s 363ms/step - loss: 0.2198 - accuracy: 0.6792
Epoch 3/10
35/35 [==============================] - 14s 365ms/step - loss: 0.2194 - accuracy: 0.6792
Epoch 4/10
35/35 [==============================] - 13s 362ms/step - loss: 0.2191 - accuracy: 0.6792
Epoch 5/10
35/35 [==============================] - 13s 364ms/step - loss: 0.2188 - accuracy: 0.6792
Epoch 6/10
35/35 [==============================] - 13s 363ms/step - loss: 0.2185 - accuracy: 0.6792
Epoch 7/10
35/35 [==============================] - 13s 364ms/step - loss: 0.2183 - accuracy: 0.6792
Epoch 8/10
35/35 [==============================] - 13s 364ms/step - loss: 0.2180 - accuracy: 0.6792
Epoch 9/10
35/35 [==============================] - 13s 361ms/step - loss: 0.2178 - accuracy: 0.6792
Epoch 10/10
35/35 [==============================] - 13s 362ms/step - loss: 0.2176 - accuracy: 0.6792


## Test Some data

In [109]:
score = completeModel.evaluate(testImages)
print ("loss = ", score[0])
print ("accuracy = ", score[1])

10/10 [==============================] - 4s 351ms/step - loss: 0.2087 - accuracy: 0.7057
loss =  0.20871223509311676
accuracy =  0.7056962251663208


## See what happens with a deer image

In [110]:
# testPicture = load_img('../deerdownload.jpg', target_size=(224,224))
testPicture = load_img('../DeerNight.jpg', target_size=(224,224))

pictureArray = img_to_array(testPicture)
# reshape data for the model
pictureArray = pictureArray.reshape((1, pictureArray.shape[0], pictureArray.shape[1], pictureArray.shape[2]))
# prepare the image for the VGG model
pictureArray = preprocess_input(pictureArray)
# load the model

# predict the probability across all output classes
yhat = completeModel.predict(pictureArray)
# ynew = completeModel.predict_classes(pictureArray)
# convert the probabilities to class labels
# labels = decode_predictions(yhat)
# retrieve the most likely result, e.g. highest probability
# label = labels[0][0]
# print the classification
print(yhat)

1/1 [==============================] - 0s 13ms/step
[[0.32589078 0.6741093 ]]
